## LightGBM  (5)  Tuning
- player情報を付加
- 2017年の実績を付加
- 特徴量数: 135

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import lightgbm as lgb
import optuna.integration.lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [18]:
ALL_MERGE = 'intermediate/all_merge_5.f'

In [19]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 150)

### 不要な列を削除

In [20]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [21]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [22]:
print(all_pitch.shape)
all_pitch.head()

(778767, 135)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,elapsed_min,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,...,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,batter_cnt_bat,bat_game_cnt_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,firm,height,weight,draft_order,salary,foreigner,batter_cnt,bat_game_cnt,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0.0,1.0,1,1,0,0,0.000000,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0,0,0,0,0.0,1.0,1,1,0,0,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0,0,0,0,0.0,1.0,1,1,0,0,0.383333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0,0,0,0,0.0,0.0,1,1,0,0,1.016667,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0,0,0,0,0.0,0.0,1,1,0,0,1.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681


### train
- 行数: 257117

In [23]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 135)

### test
- 行数: 521650

In [24]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 135)

In [25]:
del all_pitch
gc.collect()

64

In [26]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [27]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [28]:
X_train, X_test, y_train, y_test = train_test_split(train_d, train['ball'])
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

### LGBM学習
- multiclass : クラス数=8

In [29]:
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
    }

### Tuningしながら学習

In [31]:
%%time
best_params, tuning_history = dict(), list()
lgb_model = lgb.train(lgb_param, lgb_train,
                      valid_sets=lgb_eval,
                      verbose_eval=0,
                      best_params=best_params,
                      tuning_history=tuning_history)
print('Best Params:', best_params)
print('Tuning history:', tuning_history)

/Users/umeharayasunobu/opt/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: train is experimental (supported from v0.18.0). The interface can change in the future.
  ExperimentalWarning,
tune_num_leaves, val_score: 1.117045: 100%|##########| 20/20 [1:56:16<00:00, 348.83s/it]
tune_bagging_fraction_and_bagging_freq, val_score: 1.117045:  10%|#         | 1/10 [03:20<30:03, 200.38s/it][I 2020-05-03 21:47:54,627] Finished trial#0 with value: 1.122439407427727 with parameters: {'bagging_fraction': 0.6382756973834607, 'bagging_freq': 5}. Best is trial#0 with value: 1.122439407427727.
tune_bagging_fraction_and_bagging_freq, val_score: 1.117045:  20%|##        | 2/10 [07:10<27:53, 209.21s/it][I 2020-05-03 21:51:44,431] Finished trial#1 with value: 1.1188240886623797 with parameters: {'bagging_fraction': 0.8389448732985845, 'bagging_freq': 7}. Best is trial#1 with value: 1.1188240886623797.
tune_bagging_fraction_and_bagging_freq, val_score: 1.116179:  30%|##

tune_bagging_fraction_and_bagging_freq, val_score: 1.116179:  40%|####      | 4/10 [14:48<21:49, 218.32s/it][I 2020-05-03 21:59:22,792] Finished trial#3 with value: 1.1191744414195473 with parameters: {'bagging_fraction': 0.7619977484270637, 'bagging_freq': 7}. Best is trial#2 with value: 1.1161792313904824.
tune_bagging_fraction_and_bagging_freq, val_score: 1.116179:  50%|#####     | 5/10 [18:13<17:51, 214.29s/it][I 2020-05-03 22:02:47,696] Finished trial#4 with value: 1.121890943227777 with parameters: {'bagging_fraction': 0.6492675560966444, 'bagging_freq': 7}. Best is trial#2 with value: 1.1161792313904824.
tune_bagging_fraction_and_bagging_freq, val_score: 1.116179:  60%|######    | 6/10 [21:27<13:53, 208.27s/it][I 2020-05-03 22:06:01,910] Finished trial#5 with value: 1.1242426646933497 with parameters: {'bagging_fraction': 0.5209635864609548, 'bagging_freq': 4}. Best is trial#2 with value: 1.1161792313904824.
tune_bagging_fraction_and_bagging_freq, val_score: 1.116179:  70%|#####

tune_lambda_l1_and_lambda_l2, val_score: 1.111642:  70%|#######   | 14/20 [56:09<24:32, 245.34s/it][I 2020-05-03 23:40:47,094] Finished trial#13 with value: 1.1170176979435769 with parameters: {'lambda_l1': 3.6243531103892905e-05, 'lambda_l2': 0.0017525097667007645}. Best is trial#12 with value: 1.1116419546699825.
tune_lambda_l1_and_lambda_l2, val_score: 1.111642:  75%|#######5  | 15/20 [1:00:27<20:45, 249.17s/it][I 2020-05-03 23:45:05,190] Finished trial#14 with value: 1.1122137030077988 with parameters: {'lambda_l1': 9.354840227873856, 'lambda_l2': 0.00018781650111581886}. Best is trial#12 with value: 1.1116419546699825.
tune_lambda_l1_and_lambda_l2, val_score: 1.111642:  80%|########  | 16/20 [1:04:15<16:11, 242.88s/it][I 2020-05-03 23:48:53,386] Finished trial#15 with value: 1.1155738924333825 with parameters: {'lambda_l1': 0.3338192428833084, 'lambda_l2': 0.01389223836578378}. Best is trial#12 with value: 1.1116419546699825.
tune_lambda_l1_and_lambda_l2, val_score: 1.111642:  85%

Best Params: {'lambda_l1': 6.9923570049658075, 'lambda_l2': 0.002378623984798833, 'num_leaves': 18, 'feature_fraction': 0.45199999999999996, 'bagging_fraction': 0.9799724836460725, 'bagging_freq': 4, 'min_child_samples': 20}


Tuning history: [{'action': 'tune_feature_fraction', 'trial': 0, 'value': "{'feature_fraction': 0.4}", 'val_score': 1.1218754711000172, 'elapsed_secs': 221.88725399971008, 'average_iteration_time': 0.22188725399971007}, {'action': 'tune_feature_fraction', 'trial': 1, 'value': "{'feature_fraction': 0.5}", 'val_score': 1.121642181489591, 'elapsed_secs': 291.05969500541687, 'average_iteration_time': 0.2910596950054169}, {'action': 'tune_feature_fraction', 'trial': 2, 'value': "{'feature_fraction': 0.6}", 'val_score': 1.1222837051532413, 'elapsed_secs': 279.5979540348053, 'average_iteration_time': 0.2795979540348053}, {'action': 'tune_feature_fraction', 'trial': 3, 'value': "{'feature_fraction': 0.7}", 'val_score': 1.124274356519188, 'elapsed_secs': 308.3501088619232, 'average_iteration_time': 0.3083501088619232}, {'action': 'tune_feature_fraction', 'trial': 4, 'value': "{'feature_fraction': 0.8}", 'val_score': 1.1245420048346424, 'elapsed_secs': 337.481840133667, 'average_iteration_time':

CPU times: user 20h 9min 40s, sys: 9min 27s, total: 20h 19min 7s
Wall time: 5h 13min 42s


### Feature Importance

In [32]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
127,play_year_dif_b-c,1624
14,ball_cnt,1648
96,salary_x_year_bat,1670
6,player_cnt_in_game,1755
84,weight_bat,1824
83,height_bat,1855
125,bmi_dif_p-c,1877
60,curve_pit,1924
118,salary_x_year_dif_p-b,1933
63,fork_pit,1939


In [33]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 12min 22s, sys: 3.52 s, total: 12min 25s
Wall time: 3min 18s


In [34]:
predict

array([[4.04144871e-01, 3.80744334e-02, 1.40115181e-01, ...,
        2.14213542e-05, 1.98837390e-05, 1.36553824e-01],
       [3.63516378e-01, 6.87216451e-02, 1.39380494e-01, ...,
        3.70543902e-05, 1.71803590e-05, 6.54304785e-02],
       [2.91635044e-01, 8.98421272e-02, 1.60443242e-01, ...,
        1.25499556e-04, 3.36927436e-05, 9.44008636e-02],
       ...,
       [5.23146864e-01, 6.63581134e-02, 1.25336489e-03, ...,
        5.43953008e-05, 2.13146919e-05, 1.48939521e-03],
       [5.39578195e-01, 2.68859776e-02, 8.68389191e-04, ...,
        3.54516349e-05, 2.14576247e-05, 9.03745447e-04],
       [5.56383687e-01, 3.22053717e-02, 1.41482288e-03, ...,
        4.40056234e-05, 2.46491005e-05, 9.20190670e-04]])

## 結果出力

In [35]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.404145,0.038074,0.140115,0.276579,0.004491,0.000021,0.000020,0.136554
1,1,0.363516,0.068722,0.139380,0.357256,0.005641,0.000037,0.000017,0.065430
2,2,0.291635,0.089842,0.160443,0.267615,0.095905,0.000125,0.000034,0.094401
3,3,0.366847,0.108326,0.203178,0.211375,0.021876,0.000082,0.000025,0.088292
4,4,0.264393,0.069653,0.286142,0.247355,0.018344,0.000072,0.000025,0.114015


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [36]:
submit.mean()

index    260824.500000
0             0.522751
1             0.076522
2             0.178174
3             0.056225
4             0.071113
5             0.042583
6             0.007270
7             0.045362
dtype: float64

In [37]:
submit.to_csv('submit/tuning.csv', header=False, index=False)

### Tuning結果
`
Best Params: {
    'lambda_l1': 6.9923570049658075, 
    'lambda_l2': 0.002378623984798833, 
    'num_leaves': 18, 
    'feature_fraction': 0.45199999999999996, 
    'bagging_fraction': 0.9799724836460725, 
    'bagging_freq': 4, 
    'min_child_samples': 20
}
`

### 評価結果
#### tuning (135)
- 評価結果  : 1.3998383

#### 5 (135)
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806